# Plausability of Lottery Luck

This tool appraises whether it is plausible that a given individual won a set of lottery prizes honestly. 

The code reads a comma-separated values file (CSV) of wins and odds.

The user inputs the number of residents of the state, and a tiny "threshold" probability.

The code outputs a lower bound on the amount everyone in the state would have had to spend for any of them to have a tiny chance of winning so often, where "tiny" is the threshold number chosen by the user.

If the required spending amount is, for example, several times the median house price in the state, it may call into question whether the winner won honestly.

The current version can analyze data for only one gambler at a time. 

## Instructions:
1. Compile a CSV file for each gambler. The CSV file should contain three columns:  "Probability," "Number," and "Cost." 

Each row corresponds to one type of wager. "Probability" is the chance of winning that wager; "Number" is the number of times the gambler collected on that wager; and "Cost" is the cost per ticket or play on that wager.

2. Put the filename of your CSV file in the box below, along with the values of POPULATION and THRESHOLD.

3. On the toolbar of this browser window (under the jupyter logo), click "Cell" --> "Run All". Wait a bit for your results to appear at the bottom of this page. 

In [1]:
from __future__ import print_function, division

# Put the name of your CSV file here:
# CSV_FILENAME = 'FILL_ME_IN.csv'
CSV_FILENAME = 'manning-edit.csv'

# set the cutoff probability
POPULATION = 10**7   # population of North Carolina
THRESHOLD = 10**(-7) # one in ten million threshold

CUT = THRESHOLD / POPULATION # Bonferroni value of cutoff

In [2]:
import numpy as np
from scipy.special import betainc
from scipy.optimize import minimize

def binTail(p, n, t):
    return betainc(n, t - n + 1, p)

def constraintFn(p, n):
    return lambda x: np.sum(np.log(binTail(p, n, x))) - np.log(CUT)

def objectiveFn(c):
    return lambda x: np.dot(x, c)

def solve(x0, upperBoundVec, p, n, c, eps, debugMode):
    cons = ({'type': 'ineq', 'fun': constraintFn(p, n)})
    bnds = tuple((n[i], upperBoundVec[i]) for i in range(len(n)))
    return minimize(objectiveFn(c), x0, method='SLSQP', jac=(lambda x: c),
                    constraints=cons, bounds=bnds,
                    options={'disp': debugMode, 'maxiter': 10000, 'eps': eps})

def readCsv(filename):
    with open(CSV_FILENAME, 'r') as f:
        firstLine = f.readline()
        if firstLine.strip() != "Probability,Number,Cost":
            raise Exception('First line of CSV must be "Probability,Number,Cost"')
    values = np.loadtxt(filename, dtype=np.float_, delimiter=',', skiprows=1)
    values = np.atleast_2d(values)
    pValues = values[:,0]
    nValues = values[:,1]
    cValues = values[:,2]
    return (pValues, nValues, cValues)

def calculateBound(eps, debugMode):
    (p, n, c) = readCsv(CSV_FILENAME)
    that = n / p
    x0 = that / 4
    return solve(x0, that, p, n, c, eps, debugMode)

def solveProblem(tries=3, debugMode=False):
    # Try each of these epsilon values (Hessian approximation step sizes)
    epsilons = [1e-6, 1e-5, 1e-4, 1e-3]
    epsIndex = 0
    optimalValues = []
    while (len(optimalValues) == 0):
        for i in range(tries):
            optimOutput = calculateBound(epsilons[epsIndex], debugMode)
            if optimOutput['status'] == 0:
                optimalValues.append(optimOutput['fun'])
        if epsIndex == len(epsilons):
            raise Exception('Something went wrong while solving the problem. Please ask for assistance.')
        epsIndex += 1
    bestValue = np.min(optimalValues)
    print("Everyone in the population would have to spend at least ${:,} dollars to have probability {} that at least one would win so much."
          .format(np.int(bestValue),THRESHOLD))
    return bestValue

In [3]:
solveProblem()

Everyone in the population would have to spend at least $809,298 dollars to have probability 1e-07 that at least one would win so much.


809298.98386215651